
This appendix presents the pseudocode for both the classification and optimization algorithms defined in this study. 

The optimization algorithm includes the classification algorithm. Therefore to simplify this section only the optimization algorithm will be explained in detail.

###Optimization Algorithm
The optimization algorithm consists of three main subroutines: 
1. an **Initialization** function that reads in the pressure and wave data,
2. **Classification** in which CPs are assigned to classes and 
3. **Optimization** in which the classes are derived. Procedures of strategic importance are described as follows:



**INITIALIZATION**

Reads in pressure data,<br>
wave data and initial random definitions of CP classes.<br> Converts pressure values to anomalies.<br>

INPUT:

   Pressure data file,<br>
   wave data file,<br>
   CP class file,<br>
   Files containing average and standard deviations of pressure values,<br>
   number of grid points in the y-direction $Y$,<br>
   number of grid points in the x-direction $X$,<br>
   number of CP classes ($n$),<br>
   length of record $T$.
   
OUPTUT:

   Anomaly data matrix ($P \in \mathbb{R}^{T \times Y \times X}$),<br>
   Wave height vector ($W \in \mathbb{R}^{T}$),<br>
   CP Class matrix ($C \in \mathbb{R}^{n \times Y \times X}$).
    
$\underline{Variables}$:

 $C \in \mathbb{R}^{n\times Y \times X}$<br>
 $P \in \mathbb{R}^{T\times Y \times X}$<br>
 $W \in \mathbb{R}^{\period}$<br>
 $\mathbf{AVG} \in \mathbb{R}^{J \times Y \times X}$<br>
 $\mathbf{STD} \in \mathbb{R}^{J \times Y \times X}$ $J$ is the length of the year<br>
 $\mathbf{BA} \in \mathbb{R}^{Y\times X}$<br>
 $\mathbf{BS} \in \mathbb{R}^{Y\times X}$<br>
 Temp vectors of length $Y\times X$ to store values.

 $\mathbf{OPEN(File:}$ average pressure data file as $\mathbf{A})$<br>
 $\mathbf{OPEN(File:}$ standard deviation pressure data file as $\mathbf{S})$<br>
 For $i =1 \to J$  Read and store average and standard deviations of pressure data<br>
     &nbsp;$dummy \gets \mathbf{READ(A)}$ Dummy variable of time step in file<br>
     &nbsp;$dummy \gets \mathbf{READ(S)}$<br>
     &nbsp;$(\mathbf{BA}(k), k=1\to Y\times X) \gets \mathbf{READ(A},float)$<br>
     &nbsp;$(\mathbf{BS}(k), k=1\to Y\times X) \gets \mathbf{READ(S},float)$<br>
     &nbsp;$LL \gets 1$<br>
     &nbsp;For $k =1 \to \lat$<br>
         &nbsp;&nbsp;For$l =1 \to \lon$<br>
             &nbsp;&nbsp;&nbsp;$\mathbf{AVG}(k,l) \gets BA(LL)$<br>
             &nbsp;&nbsp;&nbsp;$\mathbf{STD}(k,l) \gets BS(LL)$<br>
             &nbsp;&nbsp;$LL \gets LL +1$<br>
         &nbsp;&nbsp;EndFor<br>
      &nbsp;EndFor<br>
EndFor

 $\mathbf{OPEN(File:}$ CP classes file as $\mathbf{CP})$<br>
For $i =1 \to n$ Read and store CP classes<br>
&nbsp;$dummy \gets \mathbf{READ(CP)}$ Dummy variable of class number in file<br>
$(\mathbf{BA}(k), k=1\to Y\times X) \gets \mathbf{READ(CP},int)$<br>
&nbsp;$LL \gets 1$<br>
&nbsp;&nbsp;For $k =1 \to Y$<br>
&nbsp;&nbsp;For $l =1 \to X$<br>
&nbsp;&nbsp;&nbsp;$C(k,l) \gets BA(LL)$<br>
&nbsp;&nbsp;&nbsp;$LL \gets LL +1$<br>
&nbsp;&nbsp;EndFor<br>
&nbsp;EndFor<br>
EndFor

$\mathbf{OPEN(File:}$ Pressure data file as $\mathbf{Pr})$<br>
 $\mathbf{OPEN(File:}$ Wave data file as $\mathbf{Wa})$<br>
For $i =1 \to T$ Read and store pressure and wave data
&nbsp;$dummy \gets \mathbf{READ(Pr)}$ Dummy variable of time step in file<br>
&nbsp;$(\mathbf{BA}(k), k=1\to Y\times X) \gets \mathbf{READ(Pr},float)$<br>
&nbsp;$\waves \gets \mathbf{READ(Wa},float)$<br>
&nbsp;$LL \gets 1$<br>
&nbsp;For $k =1 \to \lat$<br>
&nbsp;&nbsp;For $l =1 \to X$<br>
&nbsp;&nbsp;&nbsp;$P(k,l) \gets BA(LL)$<br>
&nbsp;&nbsp;&nbsp;$LL \gets LL +1$<br>
&nbsp;&nbsp;EndFor<br>
&nbsp;EndFor<br>
EndFor<br>
 Reading data complete now convert pressure data to anomalies<br>
For $i=1 \to T$<br>
&nbsp;$J \gets JulianDate(i)$ Convert loop variable $i$ to a Julian date<br>
&nbsp;For $k=1 \to Y$<br>
&nbsp;&nbsp;For $l=1 \to X$<br>
&nbsp;&nbsp;&nbsp;State $P(i,k,l) \gets \dfrac{\pressure(i,k,l) - \mathbf{AVG}(J,k,l)}{\mathbf{STD}(J,k,l)}$<br>
&nbsp;&nbsp;EndFor<br>
&nbsp;EndFor<br>
EndFor 

**Classification**

INPUT:

   number of CP classes ($n$),<br>
   Anomaly data matrix ($P \in \mathbb{R}^{T \times Y \times X}$),<br>
   CP Class matrix ($C \in \mathbb{R}^{n \times Y \times X}$),<br>
   $C$ contains the definitions of the CP classes derived in the optimization<br>
    number of grid points in the y-direction $Y$,<br>
    number of grid points in the x-direction $X$,<br>
    length of record $T$
    
OUTPUT:

A vector $\classvec \in \mathbb{R}^{\period}$ containing CP class labels for all time steps in $\period$.%$T$

\emph{$\underline{Variables}$}:
$\powers \in \real{4}$ user defined exponent values for fuzzy numbers $1,\ldots, 4$

 $\counts \in \real{4}$ \Comment{vector to store counts of different fuzzy numbers $1,\ldots, 4$}
\State $\dofstore \in \real{4}$ \Comment{vector to store membership grades}
\State Set all values of $\counts$ and $\dofstore$ to 0
\State $\dof$ = 1
\State $\classno$ = 1
\State $\maximum$ = 0
\For{i=1 $\to \period$}
	\For{j=1 $\to \nCPs$}
		\For{k = 1$\to \lat$}
			\For{l = 1 $\to \lon$}
				\State \emph{Fuzzy Number} $\gets$ $\classes(j,k,l)$
				\State \emph{Anomaly} $\gets$ $\pressure(i,k,l)$
				\State $grade \gets \Call{Membership}{Fuzzy Number, Anomaly}$\\
			\Comment{Calculate the membership grade for anomaly value based on the fuzzy number}
				\State $\dofstore(Fuzzy Number) \gets \dofstore(Fuzzy Number) + grade^{\powers(Fuzzy Number)}$
				\State $\counts(Fuzzy Number) \gets \counts(Fuzzy Number) +1$
			\EndFor
		\EndFor
	\For{m = 1: 4}
		\State $\dof \gets \dof \times \left(\frac{\dofstore(m)}{\counts(m)}\right)^{1/\powers(m)}$
	\EndFor
	\If{$\dof>\maximum$}
		\State $\classno \gets  i$
		\State $\maximum \gets \dof$
	\EndIf
	\EndFor
	\State $\classvec(\period) \gets \classno$
	\State Set all values of $\counts, \dofstore$ and  $\maximum$ to 0.
	\State Set $\dof$ and $\classno = 1$
\EndFor
\end{algorithmic}
\end{algorithm}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{algorithm}
\caption{Membership}
\begin{algorithmic}
\State This procedure assigns a membership grade $\in [0,1]$ to the anomaly at a grid point based on its value and the fuzzy number at the same grid point. It is a function that is called from the CLASSIFICATION function.
\Require Fuzzy Number ($\fuzzynumber$)  and the Anomaly value ($\anomaly$) 
\Ensure Membership grade $\dofstore$
\Function{Triangular}{a,x,y,z}\\
\Comment{Calculate the membership grade using a triangular function}
\State $\dofstore = \left\{\begin{array}{lc}
\dfrac{a-x}{y-x} & if x\leq a \leq x\\
\dfrac{a-z}{y-z} & if y < a \leq z\\
0 & else 
\end{array}
\right.$
\\
\Return $\dofstore$

\EndFunction
\If{$\fuzzynumber == 1$}
	\State $\dofstore \gets \Call{Triangular}{\anomaly, -\infty, -3, 0}$
\ElsIf{$\fuzzynumber == 2$}
	\State $\dofstore \gets \Call{Triangular}{\anomaly, -4, -0.85, 4}$
\ElsIf{$\fuzzynumber == 3$}
	\State $\dofstore \gets \Call{Triangular}{\anomaly, 0.25, 0.85, 4}$
\ElsIf{$\fuzzynumber == 4$}
	\State $\dofstore \gets \Call{Triangular}{\anomaly, 0, 3, +\infty}$
\EndIf
\end{algorithmic}
\end{algorithm}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{algorithm}
\caption{Optimization}
\begin{algorithmic}
\State This optimization procedure uses simulated annealing to optimize the shape of the CP classes.
\Require Number of CPs $\nCPs$, \\ Wave vector $\waves \in \real{\period}$, \\ Anomaly matrix $\pressure \in \real{\period \times \lat \times \lon}$, \\ CP classes $\classes \in \real{\nCPs \times \lat \times \lon}$, \\ grid points in the y-direction $\lat$,\\ grid points in the x-direction $\lon$, Time period $\period$.
\Ensure An optimized CP class matrix $\classes \in \real{\nCPs \times \lat \times \lon}$.
%\State \emph{$\underline{Variables}$}:
\State $\classvec \gets \Call{Classification}{\nCPs, \pressure, \classes, \lat, \lon, \period}$
\Comment{Perform initial classification}
\State $\mathbf{Ob1}$, $\mathbf{Ob2}$, $\mathbf{Ob3}$ $\gets \Call{CalcObjectiveFuncs}{\classvec,\waves}$\\
\Comment{Calculate initial objective functions}
\State $\mathbf{Temp} \gets 7.85*\nCPs$ \Comment{Randomly set initial annealing temperature}
\For{$kt = 1 \to 150$} \Comment{Start annealing}
	\For{$i =1 \to 5000$}
		\State $\mathbf{R} \gets \Call{Random}{\nCPs}$ \Comment{Randomly select a CP class}
		\State $\mathbf{IX} \gets \Call{Random}{\lon}$ \Comment{Randomly select x-location}
		\State $\mathbf{IY} \gets \Call{Random}{\lat}$ \Comment{Randomly select y-location}
		\State $\classes(R,IY,IX) \gets f \in [1,\ldots,4], f \not= \classes(R,IY,IX)$
		\\ \Comment{Change fuzzy number at location $(IX,IY)$ for class $R$}
		\State $\classvec2 \gets \Call{Classification}{\nCPs,\pressure,\classes,\lat,\lon,\period}$\\
		\Comment{Perform new classification with changed CP class}
		\State $\mathbf{Ub1}$, $\mathbf{Ub2}$, $\mathbf{Ub3}$ $\gets \Call{CalcObjectiveFuncs}{\classvec2,\waves}$
		\State $\mathbf{O} \gets \alpha\mathbf{Ob1} + \beta\mathbf{Ob2} + \gamma\mathbf{Ob3}$
		\State $\mathbf{U} \gets \alpha\mathbf{Ub1} + \beta\mathbf{Ub2} + \gamma\mathbf{Ub3}$\\
		\Comment{Calculate old and new objective functions. Weights are user defined.}
		\State $\mathbf{O} \gets w - \mathbf{O}$
		\State $\mathbf{U} \gets w - \mathbf{U}$\\
		\Comment{Change maximization scheme to minimizing}
		\If{$\mathbf{U} < \mathbf{O}$}
			\State Accept change
		\Else
			\State Accept change with probability $\exp\left(\dfrac{\mathbf{O}-\mathbf{U}}{\mathbf{Temp}}\right)$
		\EndIf
		\If{Change i accepted}
		\State $\mathbf{O1}, \mathbf{O2}, \mathbf{O3} \gets \mathbf{U1}, \mathbf{U2}, \mathbf{U3}$
		\EndIf
		\State Reduce annealing temperature.
	\EndFor
\EndFor
\end{algorithmic}
\end{algorithm}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{algorithm}
\caption{Calculate Objective Functions}
\begin{algorithmic}
\State This procedures calculates the objective functions required for the optimization. It is called in the optimization procedure.
\Require Classification $\classvec \in \real{\period}$,\\ Wave data $\waves \in \real{\period}$, \\ Period $\period$.
\Ensure Objective functions $\mathbf{O1}, \mathbf{O2}$ and $\mathbf{O3}$.
\State \emph{$\underline{Variables}$}:
\State $\mathbf{h_{CP}} \in \real{\nCPs} \gets 0$ \Comment{Frequency of CP class}
\State $\mathbf{hs_{CP}} \in \real{\nCPs} \gets 0$\Comment{Average wave height of CP class}
\State $\mathbf{hs} \gets 0$\Comment{Average wave height}
\State $\mathbf{pr_{3.5}} \in \real{\nCPs} \gets 0$\Comment{Probability of wave height exceeding 3.5~m for a given CP}
\State $\mathbf{pe_{3.5}} \gets 0$\Comment{Probability of wave height exceeding 3.5~m}
\State $\mathbf{pr_{2.5}} \in \real{\nCPs} \gets 0$\Comment{Probability of wave height exceeding 2.5~m for a given CP}
\State $\mathbf{pe_{2.5}} \gets 0$\Comment{Probability of wave height exceeding 2.5~m}

\For{$i=1 \to \period$}
	\State $cp \gets \classvec(i)$
	\State $\mathbf{h_{CP}}(cp) \gets \mathbf{h_{CP}}(cp) +1$
	\State $\mathbf{hs_{CP}}(cp) \gets \mathbf{hs_{CP}}(cp) +\waves(i)$
	\State $\mathbf{hs} \gets \mathbf{hs} +\waves(i)$
	\If{$\waves(i)\geq2.5$}
		\State $\mathbf{pr_{2.5}}(cp) \gets \mathbf{pr_{2.5}}(cp)+1$
		\State $\mathbf{pe_{2.5}} \gets \mathbf{pe_{2.5}} +1$
	\EndIf
	\If{$\waves(i)\geq3.5$}
		\State $\mathbf{pr_{3.5}}(cp) \gets \mathbf{pr_{3.5}}(cp)+1$
		\State $\mathbf{pe_{3.5}} \gets \mathbf{pe_{3.5}} +1$
	\EndIf
		
\EndFor
\State $\mathbf{hs} \gets \mathbf{hs}/\period$
\State $\mathbf{pe_{2.5}} \gets \mathbf{pe_{2.5}}/\period$
\State $\mathbf{pe_{3.5}} \gets \mathbf{pe_{3.5}}/\period$
\For{$i=1 \to \nCPs$}	
	\State $\mathbf{hs_{CP}}(i) \gets \mathbf{hs_{CP}}(i)/\mathbf{h_{CP}}(i)$
	\State $\mathbf{O1} \gets \mathbf{O1} + \mathbf{hs_{CP}}(i)/\mathbf{hs}$
	\State $\mathbf{pr_{2.5}}(i) \gets \mathbf{pr_{2.5}}(i)/\mathbf{h_{CP}}(i)$
	\State $\mathbf{O2} \gets \mathbf{O2} + (\mathbf{pr_{2.5}}(i) - \mathbf{pe_{2.5}})^2$
	\State $\mathbf{pr_{3.5}}(i) \gets \mathbf{pr_{3.5}}(i)\mathbf{h_{CP}}(i)$
	\State $\mathbf{O3} \gets \mathbf{O3} + (\mathbf{pr_{3.5}}(i) - \mathbf{pe_{3.5}})^2$
\EndFor

\end{algorithmic}
\end{algorithm}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

%\Function{Classification}{$n$, $A$, $K$}
%	\State $n$ - number of CPs
%\EndFunction


%\begin{algorithm}                      % enter the algorithm environment
%\caption{Calculate $y = x^n$}          % give the algorithm a caption
%\label{alg1}                           % and a label for \ref{} commands later in the document
%\begin{algorithmic}                    % enter the algorithmic environment
%    \Require $n \geq 0 \vee x \neq 0$
%    \Ensure $y = x^n$
%    \State $y \Leftarrow 1$
%    \Function{Classification}{$n$, $A$, $K$}
%	\State $n$ - number of CPs
%\EndFunction
%    \If{$n < 0$}
%        \State $X \Leftarrow 1 / x$
%        \State $N \Leftarrow -n$
%    \Else
%        \State $X \Leftarrow x$
%        \State $N \Leftarrow n$
%    \EndIf
%    \While{$N \neq 0$}
%        \If{$N$ is even}
%            \State $X \Leftarrow X \times X$
%            \State $N \Leftarrow N / 2$
%        \Else[$N$ is odd]
%            \State $y \Leftarrow y \times X$
%            \State $N \Leftarrow N - 1$
%        \EndIf
%    \EndWhile
%\end{algorithmic}
%\end{algorithm}

